In [1]:
import pandas as pd
import numpy as np
from pathlib import Path


In [2]:
PROJECT_ROOT = Path("..")

RAW_FILE = PROJECT_ROOT / "2016Q1.csv"
LABELS_FILE = PROJECT_ROOT / "data" / "processed" / "labels_2016Q1.parquet"

assert RAW_FILE.exists(), f"Missing raw file: {RAW_FILE}"
assert LABELS_FILE.exists(), f"Missing labels file: {LABELS_FILE}"

RAW_FILE, LABELS_FILE


(PosixPath('../2016Q1.csv'),
 PosixPath('../data/processed/labels_2016Q1.parquet'))

In [3]:
labels_df = pd.read_parquet(LABELS_FILE)
labels_df.head(), labels_df.shape


(        loan_id  months_observed  default_24m
 0  100000512540               24            0
 1  100000549022               24            0
 2  100003006281               24            0
 3  100003784303               24            0
 4  100004643080               24            0,
 (338473, 3))

In [4]:
FEATURE_COLS = {
    "loan_id": 1,
    "report_period": 2,          # MMYYYY, only used to identify first row
    "orig_interest_rate": 7,     # interest rate
    "orig_upb": 9,               # original unpaid principal balance
    "orig_loan_term": 12,        # should be 360
    "property_type": 27,         # SF, CO, PU, etc.
    "loan_purpose": 29,          # P, C, R
    "property_state": 30,        # GA, TX, IL, etc.
    "loan_type": 34              # FRM, ARM
}


In [5]:
usecols = list(FEATURE_COLS.values())

reader = pd.read_csv(
    RAW_FILE,
    sep="|",
    header=None,
    usecols=usecols,
    dtype=str,
    chunksize=200_000,
    engine="c"
)

orig_rows = []

for chunk in reader:
    chunk = chunk.rename(columns={v: k for k, v in FEATURE_COLS.items()})

    # parse reporting period to identify first record
    chunk["report_period"] = pd.to_datetime(
        chunk["report_period"], format="%m%Y", errors="coerce"
    )

    # keep first row per loan in this chunk
    firsts = (
        chunk.sort_values(["loan_id", "report_period"])
             .groupby("loan_id", as_index=False)
             .first()
    )

    orig_rows.append(firsts)

orig_df = (
    pd.concat(orig_rows, ignore_index=True)
      .sort_values(["loan_id", "report_period"])
      .groupby("loan_id", as_index=False)
      .first()
)

orig_df.head(), orig_df.shape


(        loan_id report_period orig_interest_rate   orig_upb orig_loan_term  \
 0  100000512540    2016-02-01              3.750   65000.00            360   
 1  100000549022    2016-03-01              4.000  153000.00            360   
 2  100003006281    2016-03-01              3.625  114000.00            360   
 3  100003784303    2016-03-01              3.875  150000.00            360   
 4  100004643080    2016-01-01              4.250   40000.00            360   
 
   property_type loan_purpose property_state loan_type  
 0            SF            P             GA       FRM  
 1            SF            P             KS       FRM  
 2            SF            P             IL       FRM  
 3            PU            P             IN       FRM  
 4            PU            P             TX       FRM  ,
 (405925, 9))

In [6]:
model_df = labels_df.merge(orig_df, on="loan_id", how="inner")

model_df.shape


(338473, 11)

In [7]:
model_df["orig_interest_rate"] = pd.to_numeric(model_df["orig_interest_rate"], errors="coerce")
model_df["orig_upb"] = pd.to_numeric(model_df["orig_upb"], errors="coerce")
model_df["orig_loan_term"] = pd.to_numeric(model_df["orig_loan_term"], errors="coerce")

model_df.isna().mean().sort_values(ascending=False)


loan_id               0.0
months_observed       0.0
default_24m           0.0
report_period         0.0
orig_interest_rate    0.0
orig_upb              0.0
orig_loan_term        0.0
property_type         0.0
loan_purpose          0.0
property_state        0.0
loan_type             0.0
dtype: float64

In [8]:
OUT_DIR = PROJECT_ROOT / "data" / "processed"
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_PATH = OUT_DIR / "model_data_2016Q1.parquet"
model_df.to_parquet(OUT_PATH, index=False)

OUT_PATH


PosixPath('../data/processed/model_data_2016Q1.parquet')

In [9]:
from pathlib import Path

out_dir = Path("..") / "data" / "processed"
out_dir.mkdir(parents=True, exist_ok=True)

model_df.to_parquet(out_dir / "model_data_2016Q1.parquet", index=False)

out_dir / "model_data_2016Q1.parquet"


PosixPath('../data/processed/model_data_2016Q1.parquet')